# Imports

In [2]:
from sklearn.preprocessing import RobustScaler, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
import re
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import itertools
import matplotlib.pyplot as plt
from torch.nn import MSELoss
import shap

# Data preprocessing

In [3]:
# Load the dataset
dataset = pd.read_csv('battery_feature_extracted.csv')

In [4]:
# Select features and target
X = dataset.drop(columns=['average_voltage'])
y = dataset['average_voltage']

In [5]:
# First split to separate out the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
# Second split: separate the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)  # 20% for validation


In [7]:
# Standardizing the features (fit on X_train, apply to all)
scaler = RobustScaler()
#scaler = StandardScaler()
#scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1)  # Ensure target tensor is of the right shape
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val.values).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1)

In [9]:
# Model parameters
num_features = X_train_scaled.shape[1]
output_size = 1  # For regression, we predict a single continuous value

# Define and Load the Saved Model

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# Define GRU model
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.5):
        super(GRUNetwork, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, h_n = self.gru(x)
        return self.fc(h_n[-1])


In [12]:
class TabTransformerWithGRU(nn.Module):
    def __init__(self, num_features, output_size=1, dim_embedding=128, num_heads=2, num_layers=2,
                 gru_hidden_size=128, gru_num_layers=1, gru_dropout=0.5):
        super(TabTransformerWithGRU, self).__init__()
        self.embedding = nn.Linear(num_features, dim_embedding)
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim_embedding, nhead=num_heads, dropout=0.2, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.gru_network = GRUNetwork(dim_embedding, gru_hidden_size, output_size, gru_num_layers, gru_dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = self.transformer(x)
        return self.gru_network(x)


In [13]:
# Load model
model = TabTransformerWithGRU(num_features=X_train.shape[1]).to(device)
model.load_state_dict(torch.load('entire_model_transformer_rnn_gru_mae2765_mse2943_r28877.pth', map_location=device))
model.eval()

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_20428\1320738070.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless th

TabTransformerWithGRU(
  (embedding): Linear(in_features=3226, out_features=128, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (gru_network): GRUNetwork(
    (gru): GRU(128, 128, batch_first=True, dropout=0.5)
    (fc): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [14]:
# Set the model to evaluation mode
model.eval()

TabTransformerWithGRU(
  (embedding): Linear(in_features=3226, out_features=128, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (gru_network): GRUNetwork(
    (gru): GRU(128, 128, batch_first=True, dropout=0.5)
    (fc): Linear(in_features=128, out_features=1, bias=True)
  )
)

# Generate Predictions

In [15]:
# Generate predictions
with torch.no_grad():
    predictions = model(X_test_tensor.to(device))

# Convert predictions and targets to NumPy arrays
y_pred = predictions.cpu().numpy().flatten()
y_true = y_test_tensor.numpy().flatten()

# Compute metrics
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# Print results
print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Test R²:  {r2:.4f}")

Test MSE: 0.2943
Test MAE: 0.2765
Test R²:  0.8877


# Bootstrapping + Confidence Interval

In [16]:
n_bootstraps = 1000
rng = np.random.RandomState(42)

mse_scores, mae_scores, r2_scores = [], [], []

for _ in range(n_bootstraps):
    indices = rng.choice(len(y_true), size=len(y_true), replace=True)
    y_true_boot = y_true[indices]
    y_pred_boot = y_pred[indices]
    
    mse_scores.append(mean_squared_error(y_true_boot, y_pred_boot))
    mae_scores.append(mean_absolute_error(y_true_boot, y_pred_boot))
    r2_scores.append(r2_score(y_true_boot, y_pred_boot))

In [17]:
print(f"MSE: {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")
print(f"MAE: {np.mean(mae_scores):.4f} ± {np.std(mae_scores):.4f}")
print(f"R² : {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")

MSE: 0.2869 ± 0.0747
MAE: 0.2749 ± 0.0218
R² : 0.8885 ± 0.0308


In [18]:
# Convert to NumPy arrays
mse_scores = np.array(mse_scores)
mae_scores = np.array(mae_scores)
r2_scores = np.array(r2_scores)

# Compute statistics
def summarize(metric_array, name):
    mean = np.mean(metric_array)
    std = np.std(metric_array)
    ci_lower, ci_upper = np.percentile(metric_array, [2.5, 97.5])
    print(f"{name}: {mean:.4f} ± {std:.4f} (95% CI: [{ci_lower:.4f}, {ci_upper:.4f}])")

print("\n🔁 Bootstrapped Test Metrics (95% Confidence Intervals):")
summarize(mae_scores, "MAE")
summarize(mse_scores, "MSE")
summarize(r2_scores, "R²")


🔁 Bootstrapped Test Metrics (95% Confidence Intervals):
MAE: 0.2749 ± 0.0218 (95% CI: [0.2347, 0.3198])
MSE: 0.2869 ± 0.0747 (95% CI: [0.1687, 0.4572])
R²: 0.8885 ± 0.0308 (95% CI: [0.8161, 0.9386])
